In [1]:
import torch
from torch import nn
from torch.autograd import Variable
from torch.nn import functional as F
import torch.utils.data

from torchvision.models.inception import inception_v3
from torchvision import datasets
import torchvision.transforms as transforms

from torch.utils.data import DataLoader, Subset

import numpy as np
from scipy.stats import entropy

import pandas as pd

/home/ciafa/mnt_point_3/dasilva/venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def inception_score_src(imgs, cuda=True, batch_size=32, resize=False, splits=1):
    N = len(imgs)
    dtype = torch.cuda.FloatTensor if torch.cuda.is_available() else torch.FloatTensor

    dataloader = torch.utils.data.DataLoader(imgs, batch_size=batch_size)

    inception_model = inception_v3(pretrained=True, transform_input=False).type(dtype)
    inception_model.eval()
    up = nn.Upsample(size=(299, 299), mode='bilinear').type(dtype)
    def get_pred(x):
        if resize:
            x = up(x)
        x = inception_model(x)
        return F.softmax(x, dim=1).data.cpu().numpy()

    preds = np.zeros((N, 1000))

    for i, batch in enumerate(dataloader):
        batch = batch.type(dtype)
        batchv = Variable(batch)
        batch_size_i = batch.size()[0]

        preds[i*batch_size:i*batch_size + batch_size_i] = get_pred(batchv)

    split_scores = []

    for k in range(splits):
        part = preds[k * (N // splits): (k+1) * (N // splits), :]
        py = np.mean(part, axis=0)
        scores = []
        for i in range(part.shape[0]):
            pyx = part[i, :]
            scores.append(entropy(pyx, py))
        split_scores.append(np.exp(np.mean(scores)))

    return np.mean(split_scores), np.std(split_scores)


In [3]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, orig):
        self.orig = orig

    def __getitem__(self, index):
        return self.orig[index][0]

    def __len__(self):
        return len(self.orig)

In [4]:
# configure image transformations
rm_transforms = []
#rm_transforms += [transforms.Resize(32)]
rm_transforms += [transforms.ToTensor()]
rm_transforms += [transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]


# load real data
real_dataset = datasets.ImageFolder(root='data/Alzheimer_s Dataset/train',
                                   transform=transforms.Compose(rm_transforms))
#real_dataset = datasets.ImageFolder(root='data/Alzheimer_s Dataset/train')
real_dataloader = DataLoader(dataset=real_dataset, batch_size=32)

# load synth data
synth_dataset = datasets.ImageFolder(root='data/synth',
                                    transform=transforms.Compose(rm_transforms))
synth_dataloader = DataLoader(dataset=synth_dataset, batch_size=32)

In [5]:
real_dataset

Dataset ImageFolder
    Number of datapoints: 5122
    Root location: data/Alzheimer_s Dataset/train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )

In [6]:
synth_dataset

Dataset ImageFolder
    Number of datapoints: 20000
    Root location: data/synth
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )

In [7]:
%%time
real_is = inception_score_src(CustomDataset(real_dataset), cuda=True, batch_size=64, resize=True, splits=1)

/home/ciafa/mnt_point_3/dasilva/venv/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/home/ciafa/mnt_point_3/dasilva/venv/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


CPU times: user 21 s, sys: 6.21 s, total: 27.2 s
Wall time: 16.4 s


In [8]:
%%time

synth_is = inception_score_src(CustomDataset(synth_dataset), cuda=True, batch_size=64, resize=True, splits=1)

CPU times: user 1min 30s, sys: 21.5 s, total: 1min 52s
Wall time: 59.6 s


In [9]:
inception_scores = pd.DataFrame(index=['mean', 'std'], data={'real': real_is, 'synth': synth_is})
inception_scores

,real,synth
mean,1.782476,2.267508
std,0.000000,0.000000


In [10]:
synth_is[0] / real_is[0]

1.2721115527224833

Ratio between ISs
$$ \frac{IS_{synthetic}}{IS_{real}} = 1.27$$